<a href="https://colab.research.google.com/github/Yash0411/YOLO-v3-ML/blob/master/YOLO_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#YOLO V3

#Import Modules and libraries



*   yolo_utils contains functions for reading the prediction classes from coco_classes.txt.
*   darknetmodel consist of the darknet model used.
*   utils is a file consisting functions for drawing boxes, iou, load images, etc.


In [0]:
import tensorflow as tf
from keras.models import load_model, Model
from keras.preprocessing.image import load_img, img_to_array
from yolo_utils import read_classes, read_anchors
from darknetmodel import _conv_block,make_yolov3_model,WeightReader
from utils import BoundBox,_sigmoid,decode_netout,correct_yolo_boxes,_interval_overlap,bbox_iou,do_nms,load_image_pixels,get_boxes,generate_colors,draw_boxes



1.   Get the labels.
2.   Define the anchors.
3.   Declare shape for images



In [0]:
labels = read_classes("/content/coco_classes.txt")

# Parameters used in the Dataset, on which YOLOv3 was pretrained
anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]

image_shape = (720., 1280.)    

# Download the YOLOv3 weights

Download weights from https://pjreddie.com/media/files/yolov3.weights

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

# Convert the .weights to h5 file

convert.py converts the weights from .weights file to hdf5 file.

In [0]:
!python convert.py yolov3.cfg yolov3.weights /content/yolo.h5

# Load Model

In [0]:
# load yolov3 model
from keras.models import load_model
model = load_model('/content/yolo.h5')





*   Generation of different colors for number of labels.



In [0]:
# define the expected input shape for the model
WIDTH, HEIGHT = 416, 416

# define the probability threshold for detected objects
class_threshold = 0.3

# Generate Colors
colors = generate_colors(labels)



*   Declare test Directory.



In [0]:
import os
from matplotlib import pyplot as plt
images = os.listdir('/content/test')

# Predict from test Directory


In [0]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
# define the expected input shape for the model
WIDTH, HEIGHT = 416, 416

# define the probability threshold for detected objects
class_threshold = 0.3

for file in images:
    photo_filename =  '/content/test/' + file
    
    # load picture with old dimensions
    image, image_w, image_h = load_image_pixels(photo_filename, (WIDTH, HEIGHT))
    
    # Predict image
    yhat = model.predict(image)
    
    # Create boxes
    boxes = list()
    for i in range(len(yhat)):
        # decode the output of the network
        boxes += decode_netout(yhat[i][0], anchors[i], class_threshold, HEIGHT, WIDTH)

    # correct the sizes of the bounding boxes for the shape of the image
    correct_yolo_boxes(boxes, image_h, image_w, HEIGHT, WIDTH)

    # suppress non-maximal boxes
    do_nms(boxes, 0.5)

    # define the labels (Filtered only the ones relevant for this task, which were used in pretraining the YOLOv3 model)
    labels = read_classes("/content/coco_classes.txt")
    
    # get the details of the detected objects
    v_boxes, v_labels, v_scores = get_boxes(boxes, labels, class_threshold)

    # summarize what we found
    for i in range(len(v_boxes)):

        print(v_labels[i], v_scores[i])

    
    # draw what we found
    draw_boxes(photo_filename, v_boxes, v_labels, v_scores, labels, colors)